In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

def calculate_log_returns(prices):
    """
    計算對數收益率
    
    參數:
    - prices: 價格序列
    
    返回:
    - log returns
    """
    return np.log(prices[1:] / prices[:-1])

def estimate_instantaneous_volatility(returns, window=30):
    """
    估計瞬時波動率
    
    參數:
    - returns: 收益率序列
    - window: 波動率計算窗口
    
    返回:
    - 波動率估計
    """
    return np.std(returns, axis=0, ddof=1)

def detect_big_jumps(returns, volatility, threshold_percentile=0.99):
    """
    偵測大幅度跳躍
    
    參數:
    - returns: 收益率序列
    - volatility: 波動率估計
    - threshold_percentile: 跳躍閾值百分位數
    
    返回:
    - 跳躍偵測結果
    """
    # 標準化收益率
    standardized_returns = returns / volatility
    
    # 極端值理論閾值
    threshold = stats.norm.ppf(threshold_percentile)
    
    # 跳躍偵測
    jumps = np.abs(standardized_returns) > threshold
    
    return {
        'jumps': jumps,
        'jump_dates': np.where(jumps)[0],
        'jump_magnitude': np.abs(standardized_returns[jumps])
    }

def qq_test_small_jumps(returns):
    """
    QQ檢驗用於檢測小跳躍
    
    參數:
    - returns: 收益率序列
    
    返回:
    - QQ檢驗統計量
    """
    # 理論分位數
    theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(returns)))
    
    # 經驗分位數
    empirical_quantiles = np.sort(returns)
    
    # QQ檢驗統計量
    qq_statistic = np.corrcoef(theoretical_quantiles, empirical_quantiles)[0, 1]
    
    return qq_statistic

def analyze_altcoins_impact(btc_jumps, altcoin_returns, window=5):
    """
    分析BTC跳躍對Alt coins的影響
    
    參數:
    - btc_jumps: BTC跳躍日期
    - altcoin_returns: Alt coins收益率
    - window: 事件窗口大小
    
    返回:
    - 影響分析結果
    """
    event_window_returns = []
    
    for jump_date in btc_jumps:
        # 定義事件窗口
        start = max(0, jump_date - window)
        end = min(len(altcoin_returns), jump_date + window)
        
        event_window_returns.append(altcoin_returns[start:end])
    
    # 計算平均事件視窗收益率
    avg_event_returns = np.mean(event_window_returns, axis=0)
    
    return {
        'avg_event_returns': avg_event_returns,
        'abnormal_returns': avg_event_returns - np.mean(altcoin_returns)
    }

def comprehensive_jump_analysis(btc_data, altcoins_data):
    """
    執行完整的跳躍分析
    
    參數:
    - btc_data: BTC價格數據
    - altcoins_data: Alt coins價格數據
    
    返回:
    - 分析結果
    """
    # BTC log returns
    btc_returns = calculate_log_returns(btc_data['price'])
    
    # 估計波動率
    btc_volatility = estimate_instantaneous_volatility(btc_returns)
    
    # 大跳躍檢測
    big_jumps = detect_big_jumps(btc_returns, btc_volatility)
    
    # 小跳躍QQ檢驗
    qq_statistic = qq_test_small_jumps(btc_returns)
    
    # Alt coins影響分析
    altcoins_impact = {}
    for coin_name, coin_data in altcoins_data.items():
        alt_returns = calculate_log_returns(coin_data['price'])
        altcoins_impact[coin_name] = analyze_altcoins_impact(
            big_jumps['jump_dates'], 
            alt_returns
        )
    
    return {
        'btc_jumps': big_jumps,
        'qq_statistic': qq_statistic,
        'altcoins_impact': altcoins_impact
    }



In [ ]:
# 使用範例
def main():
    # 假設已準備好高頻數據
    btc_data = load_btc_high_frequency_data()
    altcoins_data = load_altcoins_high_frequency_data()
    
    results = comprehensive_jump_analysis(btc_data, altcoins_data)
    
    # 結果分析與報告
    print_analysis_results(results)

if __name__ == "__main__":
    main()